In [ ]:
#ライブラリのインポート
import optuna
import keras.backend as K
from keras.datasets import fashion_mnist
from keras.layers import Convolution2D, Input, Dense, GlobalAveragePooling2D
from keras.models import Model
from keras.utils import to_categorical

In [2]:
#学習用データの前処理
(train_x, train_y), (test_x, test_y) = fashion_mnist.load_data()
print(train_x.shape, train_y.shape, test_x.shape, test_y.shape)

train_x = train_x.reshape(-1,28,28,1)
test_x = test_x.reshape(-1,28,28,1)
train_y = to_categorical(train_y)
test_y = to_categorical(test_y)
print(train_x.shape, train_y.shape, test_x.shape, test_y.shape)

(60000, 28, 28) (60000,) (10000, 28, 28) (10000,)
(60000, 28, 28, 1) (60000, 10) (10000, 28, 28, 1) (10000, 10)


In [3]:
def create_model(num_layer, activation, mid_units, num_filters):
    """
    num_layer : 畳込み層の数
    activation : 活性化関数
    mid_units : FC層のユニット数
    num_filters : 各畳込み層のフィルタ数
    """
    inputs = Input((28,28,1))
    x = Convolution2D(filters=num_filters[0], kernel_size=(3,3), padding="same", activation=activation)(inputs)
    for i in range(1,num_layer):
        x = Convolution2D(filters=num_filters[i], kernel_size=(3,3), padding="same", activation=activation)(x)
        
    x = GlobalAveragePooling2D()(x)
    x = Dense(units=mid_units, activation=activation)(x)
    x = Dense(units=10, activation="softmax")(x)
    
    model = Model(inputs=inputs, outputs=x)
    return model

In [4]:
def objective(trial):
    #セッションのクリア
    K.clear_session()
    
    #最適化するパラメータの設定
    #畳込み層の数
    num_layer = trial.suggest_int("num_layer", 3, 7)
    
    #FC層のユニット数
    mid_units = int(trial.suggest_discrete_uniform("mid_units", 100, 500, 100))
    
    #各畳込み層のフィルタ数
    num_filters = [int(trial.suggest_discrete_uniform("num_filter_"+str(i), 16, 128, 16)) for i in range(num_layer)]
    
    #活性化関数
    activation = trial.suggest_categorical("activation", ["relu", "sigmoid", "tanh"])
    
    #optimizer
    optimizer = trial.suggest_categorical("optimizer", ["sgd", "adam", "rmsprop"])
                                          
    model = create_model(num_layer, activation, mid_units, num_filters)
    model.compile(optimizer=optimizer,
          loss="categorical_crossentropy",
          metrics=["accuracy"])
    
    history = model.fit(train_x, train_y, verbose=0, epochs=5, batch_size=128, validation_split=0.1)
    
    #検証用データに対する正答率が最大となるハイパーパラメータを求める
    return 1 - history.history["val_acc"][-1]

In [5]:
study = optuna.create_study()

In [6]:
study.optimize(objective, n_trials=100)

[I 2018-12-04 14:40:13,507] Finished a trial resulted in value: 0.40150000031789146. Current best value is 0.40150000031789146 with parameters: {'num_layer': 3, 'mid_units': 200.0, 'num_filter_0': 32.0, 'num_filter_1': 64.0, 'num_filter_2': 112.0, 'activation': 'sigmoid', 'optimizer': 'rmsprop'}.
[I 2018-12-04 14:40:35,978] Finished a trial resulted in value: 0.15050000031789146. Current best value is 0.15050000031789146 with parameters: {'num_layer': 5, 'mid_units': 400.0, 'num_filter_0': 32.0, 'num_filter_1': 48.0, 'num_filter_2': 16.0, 'num_filter_3': 32.0, 'num_filter_4': 64.0, 'activation': 'tanh', 'optimizer': 'adam'}.
[I 2018-12-04 14:42:06,600] Finished a trial resulted in value: 0.2644999996821086. Current best value is 0.15050000031789146 with parameters: {'num_layer': 5, 'mid_units': 400.0, 'num_filter_0': 32.0, 'num_filter_1': 48.0, 'num_filter_2': 16.0, 'num_filter_3': 32.0, 'num_filter_4': 64.0, 'activation': 'tanh', 'optimizer': 'adam'}.
[I 2018-12-04 14:42:58,365] Finis

In [7]:
study.best_params

{'activation': 'relu',
 'mid_units': 400.0,
 'num_filter_0': 80.0,
 'num_filter_1': 48.0,
 'num_filter_2': 80.0,
 'num_filter_3': 96.0,
 'num_filter_4': 96.0,
 'num_filter_5': 80.0,
 'num_layer': 6,
 'optimizer': 'rmsprop'}

In [8]:
study.best_value

0.09450000031789141